# **I. Import Libraries**

**Library yang digunakan adalah Numpy, Pandas, Sklearn, Scipy, Plotly, Seaborn**

In [ ]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# For Unsupervised Clustering
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

# from sklearnex import patch_sklearn #untuk mempercepat model
from imblearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV

%matplotlib inline

# **II. Data Loading**

Bagian ini berisi proses penyiapan data sebelum dilakukan eksplorasi data lebih lanjut. Proses Data Loading dapat berupa memberi nama baru untuk setiap kolom, mengecek ukuran dataset, dll

### **B. Menyimpan Data**

Data yang sudah diakses kemudian disimpan dengan nama file **'dataset.csv'** Untuk untuk kemudian data tersebut diload

In [ ]:
from google.colab import drive
drive.mount('/content/drive') ## bisa diganti dengan mengupload data ke dalam google colab

## untuk saat ini saya menggunakan drive google, artinya dataset disimpan didrive google

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **C. *Data Loading***

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Dataset_Train/dataset.csv", error_bad_lines=False)
pd.set_option("display.max.columns",None) # digunakan untuk menampilkan keseluruhan kolom pada dataset
pd.set_option('display.max_row', 10)
data.head()

,EMPLOYEE_ID,PHONE_PROVIDER,ASSIGNMENT,BILLING_PERIOD,POSITION_ID,PRIM_RPT_CODES,DOM_NO,INT_NO,DOM_DURATION,INT_DURATION,DOM_COST,INT_COST,EMAIL_1,EMAIL_2,CLOSED,APPROVED,APPROVED_DATE,PIN_TYPE,BUSINESS_DATA_COST,PERSONAL_DATA_COST,STAGED_AT
0,911015,PBX,B,200206,C050001,CTR ... ... ... ... ...,18,0,002:20:54,000:00:00,282629,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-31T13:51:42.459Z
1,911015,PBX,B,200207,C050001,CTR ... ... ... ... ...,37,0,005:09:42,000:00:00,495482,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-31T13:51:42.459Z
2,911015,PBX,B,200208,C050001,CTR ... ... ... ... ...,35,0,003:20:30,000:00:00,386027,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-31T13:51:42.459Z
3,911015,PBX,B,200209,C050001,CTR ... ... ... ... ...,37,0,004:31:36,000:00:00,529697,0,18-MAY-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-31T13:51:42.459Z
4,911015,PBX,B,200210,C050001,CTR ... ... ... ... ...,35,0,004:20:30,000:00:00,452044,0,18-MAY-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-31T13:51:42.459Z


In [ ]:
# Assuming you already have a DataFrame named 'data' and both columns exist
data['BUSINESS_DATA_COST'] = pd.to_numeric(data['BUSINESS_DATA_COST'], errors='coerce')
data['PERSONAL_DATA_COST'] = pd.to_numeric(data['PERSONAL_DATA_COST'], errors='coerce')

In [ ]:
## Dataset -> membagi dataset menjadi 4, yaitu business_dataset, personal_dataset, dan business_clean dan personal_clean
## perbedaaan antara clean dan dataset adalah, data clean merupakan dataset yang tidak memiliki null value dan value = 0
## sementara dataset saja, mengambil data yang != 0 lalu value yang null, di isi dengan mean
business_dataset = data[(data['BUSINESS_DATA_COST']!=0)]
personal_dataset = data[(data['PERSONAL_DATA_COST']!=0)]

In [ ]:
# Tahapan ini digunakan untuk mengisi dataset business dan personal yang memiliki value null digantikan dengan value mean
mean_value_business = data['BUSINESS_DATA_COST'].mean()
mean_value_personal = data['PERSONAL_DATA_COST'].mean()
business_dataset['BUSINESS_DATA_COST'].fillna(mean_value_business, inplace=True)
personal_dataset['PERSONAL_DATA_COST'].fillna(mean_value_personal, inplace=True)

In [ ]:
business_dataset.describe()

,BILLING_PERIOD,DOM_NO,INT_NO,DOM_COST,INT_COST,BUSINESS_DATA_COST,PERSONAL_DATA_COST
count,379587.000000,379587.000000,379587.000000,3.795870e+05,3.795870e+05,3.795870e+05,21854.000000
mean,200897.932909,107.937219,1.479150,1.632097e+05,4.856489e+04,2.202218e+04,191.498124
std,406.875131,416.314685,23.465695,4.033878e+05,3.410694e+05,4.293232e+04,5570.355065
min,200204.000000,-95.000000,-8.000000,-2.984740e+05,-6.552450e+05,1.131000e+04,0.000000
25%,200607.000000,6.000000,0.000000,1.846700e+04,0.000000e+00,1.496351e+04,0.000000
50%,200902.000000,30.000000,0.000000,7.613600e+04,0.000000e+00,1.496351e+04,0.000000
75%,201110.000000,124.000000,0.000000,1.880485e+05,0.000000e+00,1.496351e+04,0.000000
max,202306.000000,69517.000000,2694.000000,3.277256e+07,2.864604e+07,3.200000e+06,300000.000000


## Drop Kolom untuk memilih hanya kolom yang memiliki korelasi dengan data target

In [ ]:
columns_to_drop = ['DOM_DURATION', 'INT_DURATION',
    'BILLING_PERIOD', 'DOM_NO', 'INT_NO', 'EMPLOYEE_ID', 'PHONE_PROVIDER', 'ASSIGNMENT','BILLING_PERIOD','POSITION_ID',
    'PRIM_RPT_CODES','EMAIL_1','EMAIL_2','CLOSED','APPROVED','APPROVED_DATE','PIN_TYPE','STAGED_AT','PERSONAL_DATA_COST'
]

# Drop kolom-kolom yang tidak diperlukan
business_dataset = business_dataset.drop(columns=columns_to_drop)

# Tampilkan lima baris pertama DataFrame setelah penghapusan
business_dataset.head()

,DOM_COST,INT_COST,BUSINESS_DATA_COST
0,282629,0,14963.505025
1,495482,0,14963.505025
2,386027,0,14963.505025
3,529697,0,14963.505025
4,452044,0,14963.505025


In [ ]:
columns_to_drop = ['DOM_DURATION', 'INT_DURATION',
    'BILLING_PERIOD', 'DOM_NO', 'INT_NO','EMPLOYEE_ID', 'PHONE_PROVIDER', 'ASSIGNMENT','BILLING_PERIOD','POSITION_ID',
    'PRIM_RPT_CODES','EMAIL_1','EMAIL_2','CLOSED','APPROVED','APPROVED_DATE','PIN_TYPE','STAGED_AT','BUSINESS_DATA_COST'
]

# Drop kolom-kolom yang tidak diperlukan
personal_dataset = personal_dataset.drop(columns=columns_to_drop)

# Tampilkan lima baris pertama DataFrame setelah penghapusan
personal_dataset.head()

,DOM_COST,INT_COST,PERSONAL_DATA_COST
0,282629,0,3412.454351
1,495482,0,3412.454351
2,386027,0,3412.454351
3,529697,0,3412.454351
4,452044,0,3412.454351


In [ ]:
business_dataset.duplicated().sum() # mengecek apakah terdapat duplikasi pada dataset yang diload

147068

In [ ]:
business_dataset.drop_duplicates(inplace = True)
personal_dataset.drop_duplicates(inplace = True)

In [ ]:
personal_dataset

,DOM_COST,INT_COST,PERSONAL_DATA_COST
0,282629,0,3412.454351
1,495482,0,3412.454351
2,386027,0,3412.454351
3,529697,0,3412.454351
4,452044,0,3412.454351
...,...,...,...
558546,1786838,696375,3412.454351
558547,3008980,2122419,3412.454351
558640,124146,0,3412.454351
558641,175110,0,3412.454351


In [ ]:
# Menghitung jumlah nilai 0 pada kolom 'BUSINESS_DATA_COST' dan 'PERSONAL_DATA_COST'
zero_count_business_data_cost = (business_dataset['BUSINESS_DATA_COST'] == 0).sum()
zero_count_personal_data_cost = (personal_dataset['PERSONAL_DATA_COST'] == 0).sum()

# Total baris dalam DataFrame
total_rows = personal_dataset.shape[0]

# Menghitung persentase nilai 0
percentage_business_data_cost = (zero_count_business_data_cost / total_rows) * 100
percentage_personal_data_cost = (zero_count_personal_data_cost / total_rows) * 100

# Membuat DataFrame untuk menyajikan hasil
result_data = pd.DataFrame({
    'Column': ['BUSINESS_DATA_COST', 'PERSONAL_DATA_COST'],
    'Zero Count': [zero_count_business_data_cost, zero_count_personal_data_cost],
    'Total Rows': [total_rows, total_rows],
    'Percentage of Zeros': [percentage_business_data_cost, percentage_personal_data_cost]
})

result_data

,Column,Zero Count,Total Rows,Percentage of Zeros
0,BUSINESS_DATA_COST,0,226953,0.0
1,PERSONAL_DATA_COST,0,226953,0.0


In [ ]:
personal_dataset.describe()

,DOM_COST,INT_COST,PERSONAL_DATA_COST
count,2.269530e+05,2.269530e+05,226953.000000
mean,2.346216e+05,7.672423e+04,6205.222398
std,4.780454e+05,4.189812e+05,18757.019572
min,-2.984740e+05,-6.552450e+05,2520.000000
25%,5.306900e+04,0.000000e+00,3412.454351
50%,1.345360e+05,0.000000e+00,3412.454351
75%,2.629720e+05,0.000000e+00,3412.454351
max,3.277256e+07,2.864604e+07,450000.000000


In [ ]:
# check missing values dalam dataset
pd.set_option('display.max_row', 21)
personal_dataset.isna().sum().sort_values(ascending=False)

DOM_COST              0
INT_COST              0
PERSONAL_DATA_COST    0
dtype: int64

In [ ]:
personal_dataset.shape

(226953, 3)

In [ ]:
personal_dataset['PERSONAL_DATA_COST'] = personal_dataset['PERSONAL_DATA_COST'].round().astype(int)
business_dataset['BUSINESS_DATA_COST'] = business_dataset['BUSINESS_DATA_COST'].round().astype(int)

In [ ]:
personal_dataset.describe().T # melihat gambaran singkat terkait dataset

,count,mean,std,min,25%,50%,75%,max
DOM_COST,226953.0,234621.611898,478045.445986,-298474.0,53069.0,134536.0,262972.0,32772562.0
INT_COST,226953.0,76724.227131,418981.247100,-655245.0,0.0,0.0,0.0,28646038.0
PERSONAL_DATA_COST,226953.0,6204.778813,18757.085619,2520.0,3412.0,3412.0,3412.0,450000.0


**Berdasarkan Heat Map diatas kita dapat melihat bahwa terdapat korelasi antara business data cost dengan int cost ataupun dom cost, walaupun secara value korelasi tersebut cenderung tidak terlalu besar**


In [ ]:
# List of datasets
datasets = [business_dataset, personal_dataset]

# Dictionary to store numerical and categorical columns for each dataset
num_columns = {}
cat_columns = {}

# Loop through datasets
for i, df in enumerate(datasets):
    # Identify numerical and categorical columns for the current dataset
    num_columns[i] = df.select_dtypes(include=np.number).columns.tolist()
    cat_columns[i] = df.select_dtypes(include=['object']).columns.tolist()

    # Print the results
    print(f'Dataset {i + 1}:')
    print('Numerical Columns   : ', num_columns[i])
    print('Categorical Columns : ', cat_columns[i])
    print('\n')

Dataset 1:
Numerical Columns   :  ['DOM_COST', 'INT_COST', 'BUSINESS_DATA_COST']
Categorical Columns :  []


Dataset 2:
Numerical Columns   :  ['DOM_COST', 'INT_COST', 'PERSONAL_DATA_COST']
Categorical Columns :  []




**pada dataset diatas tidak terdapat kolom kategorikal**


# **IV. *Data Preprocessing***

Bagian ini berisi proses penyiapan data untuk proses pelatihan model, seperti pembagian data menjadi *data inference*, transformasi data (*encoding*, dll.), dan proses-proses lain yang dibutuhkan.

## **A. *Get Data for Model Inference***

Tahapan ini digunakan untuk menyimpan beberapa dataset sebagai sample yang kemudian nantinya model yang sudah terbentuk akan diuji menggunakan model inference ini. dan untuk mengetahui apakah model yang terbuat sudah sesuai dengan kondisi nyatanya.

tahapan ini akan membuat 4 dataset yang telah terbentuk, untuk di training, dan kemudian nantinya akan membandingkan apakah dataset clean vs dataset yang sudah di isi null value dengan mean, lebih baik yang mana dalam pemodelan ini


In [ ]:
data_inf_business_dataset = business_dataset.sample(20, random_state=25) # no random state dipilih secara acak
data_inf_personal_dataset = personal_dataset.sample(20, random_state=25) # no random state dipilih secara acak

In [ ]:
X_train_bd = business_dataset.drop(data_inf_business_dataset.index) # mengeluarkan data inference dari data train untuk business dataset
X_train_pd = personal_dataset.drop(data_inf_personal_dataset.index) # mengeluarkan data inference dari data train untuk personal dataset

In [ ]:
X_train_bd.reset_index(drop=True, inplace=True)
X_train_pd.reset_index(drop=True, inplace=True)

data_inf_business_dataset.reset_index(drop=True, inplace=True)
data_inf_personal_dataset.reset_index(drop=True, inplace=True)

## ***B. Splitting Dataset***

Tahapan ini digunakan membagi Data Train yang sebelumnya telah dibuat kemudian membaginya menjadi X_train, X_test, y_train, y_test

In [ ]:
# Split between `X` (Features) and `y` (Target)
X_bd = X_train_bd.drop('BUSINESS_DATA_COST', axis=1)
y_bd = X_train_bd['BUSINESS_DATA_COST']

## personal dataset
X_pd = X_train_pd.drop('PERSONAL_DATA_COST', axis=1)
y_pd = X_train_pd['PERSONAL_DATA_COST']

X_bd

,DOM_COST,INT_COST
0,282629,0
1,495482,0
2,386027,0
3,529697,0
4,452044,0
...,...,...
232494,1786838,696375
232495,3008980,2122419
232496,124146,0
232497,175110,0


In [ ]:
# Persiapkan Data (X_train, y_train, X_test, y_test)
datasets = [X_bd, X_pd]
targets = [y_bd, y_pd]
dataset_names = ['Business Dataset', 'Personal Dataset']

# Buat DataFrame untuk menyimpan hasil evaluasi
evaluation_results = pd.DataFrame(columns=['Dataset', 'Mean Squared Error (MSE)', 'Mean Absolute Error (MAE)', 'Root Mean Squared Error (RMSE)', 'Loss'])

# Buat DataFrame untuk menyimpan hasil prediksi
predictions_results = pd.DataFrame(columns=['Dataset', 'Actual', 'Predicted'])

# Loop untuk melatih model untuk setiap dataset
for dataset, target, dataset_name in zip(datasets, targets, dataset_names):
    # Pisahkan data menjadi set pelatihan dan set validasi
    X_train, X_test, y_train, y_test = train_test_split(dataset, target, test_size=0.2, random_state=42)

    # Buat Model Neural Network
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))  # Linear activation untuk masalah regresi

    # Pilih Optimizer, Loss Function, dan Metrik Evaluasi
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])

    # Latih Model
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

    # Evaluasi Model pada Data Pengujian
    loss, mae, mse = model.evaluate(X_test, y_test)

    # Hitung RMSE dari MSE
    rmse = np.sqrt(mse)

    print(f"\n{dataset_name} - Mean Squared Error: {mse}")

    # Gunakan Model untuk Prediksi
    predictions = model.predict(X_test)

    # Tambahkan hasil prediksi ke dalam DataFrame
    predictions_df = pd.DataFrame({'Dataset': [dataset_name]*len(y_test), 'Actual': y_test, 'Predicted': predictions.flatten()})
    predictions_results = pd.concat([predictions_results, predictions_df], ignore_index=True)

    # Simpan hasil evaluasi ke dalam DataFrame
    evaluation_results = evaluation_results.append({
        'Dataset': dataset_name,
        'Mean Squared Error (MSE)': mse,
        'Mean Absolute Error (MAE)': mae,
        'Root Mean Squared Error (RMSE)': rmse,
        'Loss': loss
    }, ignore_index=True)

# Tampilkan hasil evaluasi
print("\nEVALUATION RESULTS:")
print(evaluation_results)

# Pisahkan DataFrame hasil prediksi
predictions_results_business = predictions_results[predictions_results['Dataset'] == 'Business Dataset']
predictions_results_personal = predictions_results[predictions_results['Dataset'] == 'Personal Dataset']

# Tampilkan hasil prediksi untuk Business
print("\nPREDICTION RESULTS - Business:")
print(predictions_results_business)

# Tampilkan hasil prediksi untuk Personal
print("\nPREDICTION RESULTS - Personal:")
print(predictions_results_personal)


Epoch 1/50
5813/5813 [==============================] - 17s 3ms/step - loss: 2589013760.0000 - mae: 19627.5781 - mse: 2589013760.0000 - val_loss: 2527460096.0000 - val_mae: 18518.7031 - val_mse: 2527460096.0000
Epoch 2/50
5813/5813 [==============================] - 16s 3ms/step - loss: 2423535616.0000 - mae: 18834.6172 - mse: 2423535616.0000 - val_loss: 2589629952.0000 - val_mae: 17879.0312 - val_mse: 2589629952.0000
Epoch 3/50
5813/5813 [==============================] - 16s 3ms/step - loss: 2434105600.0000 - mae: 18641.5391 - mse: 2434105600.0000 - val_loss: 2503251712.0000 - val_mae: 17992.9043 - val_mse: 2503251712.0000
Epoch 4/50
5813/5813 [==============================] - 16s 3ms/step - loss: 2403521024.0000 - mae: 18442.7305 - mse: 2403521024.0000 - val_loss: 2503859968.0000 - val_mae: 18020.9395 - val_mse: 2503859968.0000
Epoch 5/50
5813/5813 [==============================] - 16s 3ms/step - loss: 2414157824.0000 - mae: 18516.6543 - mse: 2414157824.0000 - val_loss: 2513209600

In [ ]:
predictions_results_business

,Dataset,Actual,Predicted
0,Business Dataset,14964,24635.630859
1,Business Dataset,225000,383.917023
2,Business Dataset,14964,5122.030762
3,Business Dataset,14964,5071.764648
4,Business Dataset,14964,2544.739258
...,...,...,...
46495,Business Dataset,14964,5535.568359
46496,Business Dataset,14964,2343.272217
46497,Business Dataset,14964,7658.377441
46498,Business Dataset,14964,8579.694336


In [ ]:
predictions_results_personal

,Dataset,Actual,Predicted
46500,Personal Dataset,3412,1004.770508
46501,Personal Dataset,3412,831.019531
46502,Personal Dataset,3412,1176.001831
46503,Personal Dataset,3412,620.576477
46504,Personal Dataset,3412,330.370056
...,...,...,...
91882,Personal Dataset,3412,4720.061523
91883,Personal Dataset,3412,863.666138
91884,Personal Dataset,3412,1131.227661
91885,Personal Dataset,3412,4510.690430


In [ ]:
evaluation_results

,Dataset,Mean Squared Error (MSE),Mean Absolute Error (MAE),Root Mean Squared Error (RMSE),Loss
0,Business Dataset,2.496550e+09,17818.302734,49965.492092,2.496550e+09
1,Personal Dataset,3.663427e+08,5155.973633,19140.081505,3.663427e+08
